# Import

In [1]:
import sys
import os
import datetime
import pandas as pd
sys.path.append("../")
sys.path.append("../..")

# data load
from Clust.setting import influx_setting_KETI as ins
from Clust.clust.ingestion.influx import influx_Client_v2 as influx_Client, bucket_data

# split by working time
from Clust.clust.transformation.splitDataByCondition import working

# setting common parameter : influxDB instance
db_client = influx_Client.influxClient(ins.CLUSTDataServer2)

- TimeSeries를 입력 working time / not working time 을 기준으로 데이터를 나누는 것을 보여준다.
- 아래 순서에 따라 테스트가 진행
    1. Input : 단일 Dataframe
    2. Input : 특정 Bucket의 Dataset
- Working time 을 기준으로 데이터를 나누는 모든 챕터들은 아래와 같은 진행 순서를 갖는다.
    1. Data Preparation
        - duration : 2022.02.01~2022.02.28
        - kWeather Indoor 초등학교 
    2. splitDataByCondition

# 1. Split data by working time with Dataframe

In [2]:
## 1. Data Information
bucket_name ='air_indoor_초등학교'
measurement_name = "ICW0W2000025"

## 2. Time
start_time = pd.to_datetime("2022-02-01 00:00:00")
end_time = pd.to_datetime("2022-02-28 23:59:59")

## 3. working time criteria
workingtime_criteria = {'step': [0, 9, 18, 24], 'label': ['notworking', 'working', 'notworking']}

In [3]:
data = db_client.get_data_by_time(start_time, end_time, bucket_name, measurement_name)

In [4]:
data

,in_co2,in_humi,in_noise,in_pm01,in_pm10,in_pm25,in_temp,in_voc
time,,,,,,,,
2022-02-01 00:00:00+00:00,397.0,20.0,47.0,43.0,53.0,46.0,15.5,7.0
2022-02-01 00:01:00+00:00,399.0,20.0,48.0,43.0,53.0,46.0,15.5,6.0
2022-02-01 00:02:00+00:00,407.0,20.0,48.0,43.0,53.0,46.0,15.5,6.0
2022-02-01 00:03:00+00:00,398.0,20.0,48.0,43.0,56.0,47.0,15.5,6.0
2022-02-01 00:04:00+00:00,401.0,20.0,48.0,43.0,55.0,46.0,15.5,7.0
...,...,...,...,...,...,...,...,...
2022-02-28 23:55:00+00:00,404.0,24.0,46.0,21.0,26.0,22.0,18.5,6.0
2022-02-28 23:56:00+00:00,403.0,24.0,46.0,21.0,26.0,22.0,18.5,5.0
2022-02-28 23:57:00+00:00,400.0,24.0,46.0,21.0,25.0,22.0,18.5,7.0


## 1-4. Split Data by working time

In [5]:
split_data_result = working.get_split_data_by_workingtime_from_dataframe(data, workingtime_criteria)

In [6]:
split_data_result

{'working':                            in_co2  in_humi  in_noise  in_pm01  in_pm10  \
 time                                                                     
 2022-02-03 09:00:00+00:00   392.0     18.0      48.0     15.0     17.0   
 2022-02-03 09:01:00+00:00   399.0     18.0      48.0     16.0     18.0   
 2022-02-03 09:02:00+00:00   398.0     18.0      48.0     16.0     19.0   
 2022-02-03 09:03:00+00:00   395.0     18.0      48.0     16.0     18.0   
 2022-02-03 09:04:00+00:00   395.0     18.0      48.0     16.0     18.0   
 ...                           ...      ...       ...      ...      ...   
 2022-02-28 17:55:00+00:00   405.0     23.0      48.0     10.0     12.0   
 2022-02-28 17:56:00+00:00   403.0     23.0      48.0     11.0     13.0   
 2022-02-28 17:57:00+00:00   401.0     23.0      48.0     11.0     13.0   
 2022-02-28 17:58:00+00:00   402.0     23.0      48.0     10.0     14.0   
 2022-02-28 17:59:00+00:00   401.0     23.0      48.0     10.0     15.0   
 
            

In [7]:
split_data_result.keys()

dict_keys(['working', 'notworking'])

# 2. Split data by working time with Dataframe

In [8]:
## 1. Data Information
bucket_name = "air_indoor_초등학교"

## 2. Time
start_time = pd.to_datetime("2022-02-01 00:00:00")
end_time = pd.to_datetime("2022-02-28 23:59:59")

## 3. working time criteria
workingtime_criteria = {'step': [0, 9, 18, 24], 'label': ['notworking', 'working', 'notworking']}

In [9]:
dataset = bucket_data.get_all_msData_in_oneBucket(start_time, end_time, db_client, bucket_name)

## 2-2. Split Data by working time

In [10]:
split_dataset_result = working.get_split_data_by_workingtime_from_dataset(dataset, workingtime_criteria)

In [11]:
split_dataset_result

{'ICW0W2000022': {'working':                            in_co2  in_humi  in_noise  in_pm01  in_pm10  \
  time                                                                     
  2022-02-03 09:00:00+00:00   461.0     25.0      48.0     11.0     11.0   
  2022-02-03 09:01:00+00:00   461.0     25.0      48.0     10.0     11.0   
  2022-02-03 09:02:00+00:00   460.0     25.0      48.0     10.0     11.0   
  2022-02-03 09:03:00+00:00   463.0     25.0      49.0     10.0     11.0   
  2022-02-03 09:04:00+00:00   457.0     25.0      48.0     11.0     12.0   
  ...                           ...      ...       ...      ...      ...   
  2022-02-28 17:55:00+00:00   493.0     28.0      48.0     12.0     17.0   
  2022-02-28 17:56:00+00:00   501.0     28.0      48.0     11.0     18.0   
  2022-02-28 17:57:00+00:00   500.0     28.0      48.0     11.0     17.0   
  2022-02-28 17:58:00+00:00   503.0     28.0      48.0     11.0     15.0   
  2022-02-28 17:59:00+00:00   505.0     28.0      48.0     12

In [12]:
split_dataset_result.keys()

dict_keys(['ICW0W2000022', 'ICW0W2000023', 'ICW0W2000024', 'ICW0W2000025', 'ICW0W2000030', 'ICW0W2000031', 'ICW0W2000032', 'ICW0W2000033', 'ICW0W2000034'])

In [13]:
split_dataset_result["ICW0W2000022"]

{'working':                            in_co2  in_humi  in_noise  in_pm01  in_pm10  \
 time                                                                     
 2022-02-03 09:00:00+00:00   461.0     25.0      48.0     11.0     11.0   
 2022-02-03 09:01:00+00:00   461.0     25.0      48.0     10.0     11.0   
 2022-02-03 09:02:00+00:00   460.0     25.0      48.0     10.0     11.0   
 2022-02-03 09:03:00+00:00   463.0     25.0      49.0     10.0     11.0   
 2022-02-03 09:04:00+00:00   457.0     25.0      48.0     11.0     12.0   
 ...                           ...      ...       ...      ...      ...   
 2022-02-28 17:55:00+00:00   493.0     28.0      48.0     12.0     17.0   
 2022-02-28 17:56:00+00:00   501.0     28.0      48.0     11.0     18.0   
 2022-02-28 17:57:00+00:00   500.0     28.0      48.0     11.0     17.0   
 2022-02-28 17:58:00+00:00   503.0     28.0      48.0     11.0     15.0   
 2022-02-28 17:59:00+00:00   505.0     28.0      48.0     12.0     16.0   
 
            